# Sentdex Machine Learning Part 2

In [28]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
%matplotlib inline
import quandl    # lowercase works in Python3
import math
from sklearn import preprocessing, svm # This library allows for scaling on the features
#cross validation will shuffle the data for you
#svm also 

from sklearn.model_selection import cross_validate # the old way did not work but this will.
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [8]:
quandl.ApiConfig.api_key = '4eUGe2cSLWryQSrHxNme'
NASDAQ_DF = quandl.get('NASDAQOMX/XQC')

In [9]:
NASDAQ_DF.head()

,Index Value,High,Low,Total Market Value,Dividend Market Value
Trade Date,,,,,
2016-12-19,4934.85,0.0,0.0,0.0,0.0
2016-12-20,4953.80,0.0,0.0,0.0,0.0
2016-12-21,4948.90,0.0,0.0,0.0,0.0
2016-12-22,4934.39,0.0,0.0,0.0,0.0
2016-12-23,4940.02,0.0,0.0,0.0,0.0


In [10]:
quandl.ApiConfig.api_key = '4eUGe2cSLWryQSrHxNme'
googl_DF = quandl.get('WIKI/GOOGL')

In [11]:
googl_DF.head()  # This gives me google time series data

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0,0.0,1.0,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0,0.0,1.0,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0,0.0,1.0,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0,0.0,1.0,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0,0.0,1.0,52.542193,54.167209,52.100830,53.164113,9188600.0


In [12]:
goog1_df = googl_DF[['Adj. Open','Adj. High','Adj. Low','Adj. Close','Adj. Volume']]
goog1_df['HL_PCT'] = (googl_DF['Adj. High']-googl_DF['Adj. Close'])/googl_DF['Adj. Close']*100.0
goog1_df['PCT_change'] = (googl_DF['Adj. Close']-googl_DF['Adj. Open'])/googl_DF['Adj. Open']*100.0
goog1_df = goog1_df[['Adj. Close','HL_PCT','PCT_change','Adj. Volume']]
print(goog1_df.head())


c:\users\dicei\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\dicei\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


            Adj. Close    HL_PCT  PCT_change  Adj. Volume
Date                                                     
2004-08-19   50.322842  3.712563    0.324968   44659000.0
2004-08-20   54.322689  0.710922    7.227007   22834300.0
2004-08-23   54.869377  3.729433   -1.227880   18256100.0
2004-08-24   52.597363  6.417469   -5.726357   15247300.0
2004-08-25   53.164113  1.886792    1.183658    9188600.0


In [14]:
# moving forward on the regression problem, the goog1_df is the dataframe to use.
# Feature or Label...
forecast_col = 'Adj. Close'
goog1_df.fillna(-99999, inplace = True)
# Typically you use regression to forecast out so
forecast_out = int(math.ceil(0.01*len(goog1_df)))   #this project outs the 1 (or 0.1) percent of dataframe....many days 
goog1_df['label']= goog1_df[forecast_col].shift(-forecast_out)
goog1_df.dropna(inplace = True)
print(goog1_df.head(10))

print(goog1_df.tail(10))


            Adj. Close    HL_PCT  PCT_change  Adj. Volume      label
Date                                                                
2004-08-19   50.322842  3.712563    0.324968   44659000.0  69.639972
2004-08-20   54.322689  0.710922    7.227007   22834300.0  69.078238
2004-08-23   54.869377  3.729433   -1.227880   18256100.0  67.839414
2004-08-24   52.597363  6.417469   -5.726357   15247300.0  68.912727
2004-08-25   53.164113  1.886792    1.183658    9188600.0  70.668146
2004-08-26   54.122070  0.037068    2.820391    7094800.0  71.219849
2004-08-27   53.239345  2.326896   -1.803885    6211700.0  72.278116
2004-08-30   51.162935  3.411430   -3.106003    5196700.0  74.810934
2004-08-31   51.343492  1.308977    0.048866    4917800.0  74.199045
2004-09-01   50.280210  2.713217   -2.385589    9138200.0  70.462511
            Adj. Close    HL_PCT  PCT_change  Adj. Volume    label
Date                                                              
2017-12-01     1025.07  1.187236   -0.

# Machine Learning Regression Training and Testing - Practical Machine Learning

# now we will pick Features and Labels 

In [15]:
# Features ( -1 to 1) => preprocessing  
# Lesson 3 on Machine learning linear regression 
# Featuere drives the Label (to be predicted)


# convention: feature = Capital X, Label = lower case y

In [22]:
X = np.array(goog1_df.drop(['label'],1))  # Feature
y = np.array(goog1_df.label)# Label
X = preprocessing.scale(X)
#X = X[:-forecast_out+1]
#goog1_df.dropna(inplace = True)
y = np.array(goog1_df.label)
print(len(X), len(y))

# Now we are ready to create a training data set

3355 3355


In [30]:
print("Now creating training data set")
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)
clf = LinearRegression()   # if you want to run this paralelly in a cluster you can put LinearRegression(n_jobs = 10) 
#if I were to add the n_jobs = 10, it will split the work in 10 servers.  The training set will be much faster for ML
clf.fit(X_train, y_train)
accuracy = clf.score(X_test,y_test)
print(accuracy)

Now creating training data set
0.9781362214881201


# what if we wanted to interchange it with SVM from linear regression?

In [31]:
clf1 = svm.SVR()
clf1.fit(X_train, y_train)
accuracy1 = clf1.score(X_test,y_test)
print(accuracy1)

0.7998458551521953
